In [ ]:
### Data
import xarray as xr
from netCDF4 import Dataset
import numpy as np
from wrf import getvar, interplevel, smooth2d

### Plotting
import matplotlib.pyplot as plt
from matplotlib.cm import get_cmap
import matplotlib.ticker as mticker
import cartopy.crs as crs
from cartopy.feature import NaturalEarthFeature
import matplotlib.transforms as mtransforms

### Warnings
import warnings
warnings.filterwarnings('ignore')


In [ ]:
### Parameters
nx = 400
ny = 250
nz = 44
ti = 16
lev = 500

### Read in the data
ctl = Dataset('/p/work1/lloveras/nov2018/30km_files/ctl/wrfout_d01_2018-11-13_12_00_00')
adj = Dataset('/p/work1/lloveras/nov2018/30km_files/adj_full/wrfout_d01_2018-11-13_12_00_00')
anl = Dataset('/p/work1/lloveras/nov2018/30km_files/gfs/wrfin_d01_2018-11-15_12_00_00')

### Latitude and longitude
lats = np.asarray(getvar(ctl,'lat',timeidx=ti))
lons = np.asarray(getvar(ctl,'lon',timeidx=ti))
lons[lons >= 0] -= 360

### Pressure
p_ctl = np.asarray(getvar(ctl,'pressure',timeidx=ti))
p_adj = np.asarray(getvar(adj,'pressure',timeidx=ti))
p_anl = np.asarray(getvar(anl,'pressure'))

### Height
z_ctl = np.asarray(getvar(ctl,'z',timeidx=ti))
z_adj = np.asarray(getvar(adj,'z',timeidx=ti))
z_anl = np.asarray(getvar(anl,'z'))

### Interpolate
zl_ctl = interplevel(z_ctl, p_ctl, lev)
zl_adj = interplevel(z_adj, p_adj, lev)
zl_anl = interplevel(z_anl, p_anl, lev)


In [ ]:
### Response function box

coamps_dir = '/p/work1/lloveras/nov2018/coamps_files/'

nx_c = 301
ny_c = 201
nz_c = 45

rf = np.fromfile(coamps_dir + 'rfbox1_sig_028485_000010_1a0301x0201_2018111312_00000000_fcstfld', dtype='>f4')
rf = np.roll(np.flip(np.reshape(rf, [nz_c, ny_c, nx_c]),axis=0),-4,axis=-1)

lats_c = np.fromfile(coamps_dir + 'latitu_sfc_000000_000000_1a0301x0201_2018111312_00000000_fcstfld', dtype='>f4')
lons_c = np.fromfile(coamps_dir + 'longit_sfc_000000_000000_1a0301x0201_2018111312_00000000_fcstfld', dtype='>f4')
lats_c = np.reshape(lats_c, [ny_c, nx_c])
lons_c = np.reshape(lons_c, [ny_c, nx_c])
lons_c[lons_c >= 0] -= 360

coamps_dom = np.zeros(np.shape(lons_c))
coamps_dom[1:-1,1:-1] = 1.


In [ ]:
### Contour intervals
zlvls = np.arange(0,1500,15)
dzlvls = np.asarray([-9,-7,-5,-3,-1,1,3,5,7,9])*15

### Cartopy projection
proj = crs.LambertConformal(central_longitude=-100,standard_parallels=(20,70))

fig, axd = plt.subplot_mosaic([['left','cbar']],
                              constrained_layout=True, figsize=(6.8,3.9), dpi=200, 
                              gridspec_kw={'width_ratios':[1,0.025],'height_ratios':[1]},
                              per_subplot_kw={'left':{'projection':proj}})

##############
### LEFT
#############

# Add the gridlines
gls = axd['left'].gridlines(draw_labels=False, dms=True, x_inline=False, y_inline=False,linestyle='dashed')
gls.xlocator = mticker.FixedLocator([-180,-160, -140, -120, -100, -80, -60, -40,-20])
gls.ylocator = mticker.FixedLocator([0, 20, 40, 60, 80])

gls2 = axd['left'].gridlines(draw_labels=True, dms=True, x_inline=False, y_inline=False,linestyle='dashed')
gls2.xlocator = mticker.FixedLocator([-180,-160, -140, -120, -100, -80, -60, -40,-20])
gls2.ylocator = mticker.FixedLocator([0, 20, 40])
gls2.top_labels=False
gls2.right_labels=False
gls2.rotate_labels = False

cs1 = axd['left'].contour(lons, lats, smooth2d(zl_anl,1)/10.,levels=zlvls,
                colors='k',transform=crs.PlateCarree())
axd['left'].clabel(cs1,fmt='%1.0f',inline=1,levels=zlvls,fontsize=6,colors='k')

cs2 = axd['left'].contour(lons, lats, smooth2d(zl_ctl,1)/10.,levels=zlvls,
                colors='magenta',transform=crs.PlateCarree())
axd['left'].clabel(cs2,fmt='%1.0f',inline=1,levels=zlvls,fontsize=6,colors='magenta')

im1 = axd['left'].contourf(lons, lats, zl_ctl - zl_anl, levels=dzlvls,
             cmap=get_cmap('RdBu_r'), extend='both',
             transform=crs.PlateCarree())

axd['left'].contour(lons_c-1, lats_c, rf[25], levels=[0,1],colors='g',transform=crs.PlateCarree())
axd['left'].contour(lons_c, lats_c, coamps_dom, levels=[0,1],colors='k', linestyles='dotted',transform=crs.PlateCarree())

# Download and add the states and coastlines
states = NaturalEarthFeature(category='cultural', scale='50m',
                             facecolor='none',
                             name='admin_1_states_provinces')
axd['left'].add_feature(states, linewidth=.5, edgecolor='k',alpha=0.5)
axd['left'].coastlines('50m', linewidth=0.8,color='k',alpha=0.5)

### SET THE COLORBAR AND SHOW
cbar = fig.colorbar(im1, orientation='vertical', cax=axd['cbar'])
axd['left'].set_title('1200 UTC 15 Nov 2018 ($t$ = 48 h)')

plt.savefig('/p/work1/lloveras/nov2018/figs_pdf/fig03_wrf_coamps_domains.pdf',bbox_inches='tight')
plt.show()
